In [21]:
import pandas as pd
import numpy as np
from excel_scripts import *

# input_table = pd.read_excel("test_data.xlsx")


excel = Excel_scripts("Book1.xlsx")
excel.find_all_tables()

input_table = excel.table_dict['table_1'][0]
input_table

Class initialized
Found 4 tables.


,column1,column12,column13,column14
1,1,1,1,1
2,2,2,2,2
3,3,3,3,2
4,4,4,4,4
5,5,5,5,5


In [22]:
flattened_values = input_table.values.flatten()
column_names = input_table.columns.tolist()
flattened_vector = np.concatenate((column_names, flattened_values))
print(flattened_vector)

# encoded_table = pd.get_dummies(input_table)
# flattened_vector = encoded_table.values.flatten()

['column1' 'column12' 'column13' 'column14' 1 1 1 1 2 2 2 2 3 3 3 2 4 4 4
 4 5 5 5 5]


In [23]:
from transformers import BertModel, BertTokenizer
import torch

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Convert flattened vector to a string
vector_text = ' '.join(str(value) for value in flattened_vector)

# Tokenize the text
tokens = tokenizer.encode(vector_text, add_special_tokens=True)

# Convert tokens to PyTorch tensor
input_ids = torch.tensor(tokens).unsqueeze(0)

# Generate BERT embeddings
with torch.no_grad():
    outputs = model(input_ids)
    embeddings = outputs[0]

# Access the embeddings
embeddings = embeddings.squeeze(0)
print(embeddings)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[-0.2558, -0.0806,  0.2396,  ..., -0.3140,  0.4616,  0.6955],
        [-0.1735, -0.2991,  1.0185,  ..., -0.2490,  0.1516, -0.3882],
        [ 0.2921,  0.4320, -0.2579,  ..., -0.4466,  0.1332,  0.1381],
        ...,
        [-0.8838,  0.2932,  0.7961,  ...,  0.2386,  0.4085,  0.0985],
        [-1.0348, -0.4963,  0.3162,  ...,  0.0700,  0.5913, -0.3144],
        [ 0.6404,  0.0810, -0.0117,  ...,  0.2083, -0.5418, -0.4211]])


In [2]:
import torch
import torch.nn as nn
from transformers import BertModel

class ClassificationNet(nn.Module):
    def __init__(self, num_classes):
        super(ClassificationNet, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids):
        _, pooled_output = self.bert(input_ids)
        output = self.fc(pooled_output)
        return output

# Example usage
num_classes = 2  # Number of classes for classification (e.g., 2 for binary classification)
input_ids = torch.tensor([1, 2, 3, 4, 5])  # Example input tensor

model = ClassificationNet(num_classes)
outputs = model(input_ids)

print(outputs)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ValueError: not enough values to unpack (expected 2, got 1)